# Safe dict reading

define a function `safe_dict(d, k)` that takes in a python dict `d` and a key `k` and makes it safe to read even with keys that aren't in the dictionary. If you try to read from the dictionary with a bad key, it should return 0 instead.

```
d = {1 : 2, 3 : 4}
safe_dict(d, 1) -> 2
safe_dict(d, 'cat') -> 0
```

In [ ]:
def safe_dict(x, y):
    if y in x:
        return x[y]
    else:
        return 0

d = {1 : 2, 3 : 4}

print(safe_dict(d, 1))
print(safe_dict(d, 'cat'))
    

# File Reading: Hamlet Exercises

Open `hamlet.txt` in the `data` folder

### 1. Mentionned Hamlet

How many times is hamlet mentioned in the book?

Use python and line iteration to count it up

In [ ]:
with open('data\hamlet.txt') as f:
    contents = f.read()
    count = contents.count("HAMLET") + contents.count("Hamlet")

print(count)

### 2. File Reading as a .py program

Make a python file that defines a function that counts the number of times hamlet is mentionned using the code in the previous exercise.

Then import it in your notebook and call it here.

In [ ]:
text =  open('ProgramText.txt', 'w')

text.write("""
    with open('data\hamlet.txt') as f:
    contents = f.read()
    count = contents.count("HAMLET") + contents.count("Hamlet")

print(count)
    """)
text.close

text = open('ProgramText.txt', 'r')
out = text.read()
text.close()

print(out)



### 3. Unique words in hamlet

Write a program that counts the unique words in hamlet.

In [ ]:
print(len(set(word for word in open('data\hamlet.txt', 'r').read().split())))

# File Reading 2: A Python library.

In the `data` folder, you will find a folder called `csrgraph` which is a python library.

### 1. File count

Count the `py` files in the library using the `os` package

In [37]:
import os

len(os.listdir('data\csrgraph'))

8

### 2. For the following packages, count the number of files that import them:

- pandas 

- numpy

- numba

In [74]:
import os

packages = ['pandas', 'numpy', 'numba']
for package in packages:
    count = 0
    for filename in os.listdir('data\csrgraph'):
        with open(os.path.join('data\csrgraph', filename), 'r') as file:
            for line in file.readlines():
                if package in line:
                    count += 1
                    break
    print(package, count)

pandas 4
numpy 6
numba 6


# First NLP Program: IDF

Given a list of words, the the inverse document frequency (IDF) is a basic statistic of the amount of information of each word in the text.

The IDF formulat is:

$$IDF(w) = ln(\dfrac{N}{1 + n(w)})$$

Where:

- $w$ is the token (unique word),
- $n(w)$ is the number of documents that $w$ occurs in,
- $N$ is the total number of documents

Write a function, `idf(docs)` that takes in a list of lists of words and returns a dictionary  `word -> idf score`

Example:

```
IDF([['interview', 'questions'], ['interview', 'answers']]) -> {'questions': 0.0, 
                                                                'interview': -0.4, 
                                                                'answers': 0.0}


```

In [123]:
import numpy as np

def idf(docs):
    N = len(docs)
    results = {}
    words = []
    for i in docs:
        for j in i:
            words.append(j)
    result = set(words)

    for i in words:
        nw = 0
        for j in docs:
            if i in j:
                nw +=1

        results[i] = np.log(N/(1+nw))
        
    return(results)


idf([['interview', 'questions'], ['interview', 'answers']])

{'interview': -0.40546510810816444, 'questions': 0.0, 'answers': 0.0}

In [ ]:
import numpy as np

def idf(docs):
    d = set(docs[0]+ docs[1])
    result = {}
    N = len(docs)
    nw = 0
    
    for w in d:
        if w in docs[0]:
            nw =+ 1
        if w in docs[1]:
            nw =+ 1
        result[w] = np.log(N/(1+nw))
    return result


idf([['interview', 'questions'], ['interview', 'answers']])

# Stretch Goal: TF-IDF on Hamlet

The TF-IDF score is a commonly used statistic for the importance of words. Its $\frac{TF}{IDF}$ where TF is the "term frequency" (eg. how often the words happens in the document).

Calculate the TF-IDF dictionary on the Hamlet book.

What's the TF-IDF of "Hamlet"?

What's the word with the highest TF-IDF in the book?

# Stretch Goal: Speaker count

Use a regular expression and looping over the `hamlet.txt` file to build a dictionary `character_name -> # times speaking`.

Who speaks the most often? Who speaks the least often?